In [28]:
%load_ext autoreload
%autoreload 2

import logging
import os


wd = os.path.abspath("__file__").replace("/__file__", "").replace("notebooks", "")
os.chdir(wd)

from datetime import datetime, timedelta
from logging.handlers import TimedRotatingFileHandler


from krx.api import Krx_main_api
from krx.processing import upload_func
from krx.utils import get_jinja_yaml_conf, create_db_engine, Postgres_connect

now = datetime.now()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
conf = get_jinja_yaml_conf('./conf/api.yml', './conf/logging.yml')
api_start_date = datetime.strptime(conf['api_start_date'], '%Y-%m-%d').date()
end_date = datetime.now().date() 

# logger 설정
logger = logging.getLogger('main')
logging.basicConfig(level=eval(conf['logging']['level']),
    format=conf['logging']['format'],
    handlers = [TimedRotatingFileHandler(filename =  conf['logging']['file_name'],
                                   when=conf['logging']['when'],
                                   interval=conf['logging']['interval'],
                                   backupCount=conf['logging']['backupCount']), logging.StreamHandler()]
                   )
    

In [33]:
# Only for notebooks
import re

os.environ['_ts'] = datetime.astimezone(datetime.now()).strftime('%Y-%m-%d %H:%M:%S %z')

with open('./conf/credentials', "r") as file:
    # 각 라인 읽기
    for line in file:
        # 주석(#) 또는 빈 줄은 무시
        if line.strip() == '' or line.startswith('#'):
            continue

        # 각 라인을 '='를 기준으로 key와 value로 분리
        key, value = line.strip().split('=', 1)

        # $ENV 형식의 환경변수가 있을 경우 해당 값을 가져와서 설정
        env_var_pattern = re.compile(r'\$(\w+)')
        matches = env_var_pattern.findall(value)
        for match in matches:
            value = value.replace(f"${match}", os.environ.get(match, "")).replace('"', '')

        # 환경변수로 설정
        os.environ[key] = value

In [34]:


engine = create_db_engine(os.environ)
postgres_conn = Postgres_connect(engine)

2024-02-10 10:59:46,287 (utils.py 53) INFO ::: Connect to 172.20.10.3. DB_NAME is stocks


# 1. 지수

In [8]:
krx_api = Krx_main_api(auth_key = os.environ['auth_key'])

2024-02-10 10:49:07,383 (api.py 48) INFO ::: ### idx api is initialized! ###


In [35]:
date = postgres_conn.get_max_col(schema_name = conf['schema_name'],
                          table_name = conf[sub_url][data_name]['table_name'],
                          colname = conf[sub_url][data_name]['date_col'],
                          where = [f"{conf[sub_url][data_name]['class_col']} = '{conf[sub_url][data_name]['class_name']}'"])

date = date + timedelta(days = 1) if date else conf['api_start_date']

In [37]:
krx_api.get_data?

Signature:
krx_api.get_data(
    sub_url,
    detail_url,
    date,
    rename=None,
    _ts=datetime.datetime(2024, 2, 10, 10, 48, 37, 468638),
    skip_weekend=True,
)
Docstring: <no docstring>
File:      /projects/stocks/krx/krx/api.py
Type:      method

In [39]:
for data_name in conf[sub_url].keys():
    date = postgres_conn.get_max_col(schema_name = conf['schema_name'],
                          table_name = conf[sub_url][data_name]['table_name'],
                          colname = conf[sub_url][data_name]['date_col'],
                          where = [f"{conf[sub_url][data_name]['class_col']} = '{conf[sub_url][data_name]['class_name']}'"]) + timedelta(days = 1)

    date = postgres_conn.get_max_col(schema_name = conf['schema_name'],
                              table_name = conf[sub_url][data_name]['table_name'],
                              colname = conf[sub_url][data_name]['date_col'],
                              where = [f"{conf[sub_url][data_name]['class_col']} = '{conf[sub_url][data_name]['class_name']}'"])
    
    date = date + timedelta(days = 1) if date else conf['api_start_date']
    
    data = krx_api.get_data(sub_url, detail_url = conf[sub_url][data_name]['detail_url'], 
                     rename = conf[sub_url][data_name]['rename'], 
                     date = date,
                     _ts = os.environ['_ts'],
                     skip_weekend = conf[sub_url][data_name]['skip_weekend'])

    postgres_conn.insert_db(data, schema_name = conf['schema_name'],
                          table_name = conf[sub_url][data_name]['table_name'])

2024-02-10 11:01:27,779 (api.py 53) INFO ::: ### drv.fut_bydd_trd data in 2024-02-09 call starts! ###
2024-02-10 11:01:27,826 (api.py 62) WARNING ::: data from api is empty.
2024-02-10 11:01:27,828 (utils.py 115) INFO ::: Upload data successfully (rows: 0).
2024-02-10 11:01:27,830 (api.py 53) INFO ::: ### drv.eqsfu_stk_bydd_trd data in 2024-02-09 call starts! ###
2024-02-10 11:01:27,887 (api.py 62) WARNING ::: data from api is empty.
2024-02-10 11:01:27,888 (utils.py 115) INFO ::: Upload data successfully (rows: 0).
2024-02-10 11:01:27,890 (api.py 53) INFO ::: ### drv.eqkfu_ksq_bydd_trd data in 2024-02-09 call starts! ###
2024-02-10 11:01:27,938 (api.py 62) WARNING ::: data from api is empty.
2024-02-10 11:01:27,939 (utils.py 115) INFO ::: Upload data successfully (rows: 0).
2024-02-10 11:01:27,941 (api.py 53) INFO ::: ### drv.opt_bydd_trd data in 2024-02-09 call starts! ###
2024-02-10 11:01:27,990 (api.py 62) WARNING ::: data from api is empty.
2024-02-10 11:01:27,991 (utils.py 115) I

In [5]:
sub_url = 'drv'
for data_name in conf[sub_url].keys():
    upload_func(sub_url,
                data_name, 
               krx_api, 
               postgres_conn, 
               conf, 
               api_start_date, 
               _ts = os.environ['_ts'], 
               skip_weekend = conf[sub_url][data_name]['skip_weekend']) 

2024-02-05 00:52:29,847 (processing.py 9) INFO ::: drv.future_market upload starts!
2024-02-05 00:52:29,857 (conn.py 71) INFO ::: ### drv.fut_bydd_trd data in 2024-02-03 call starts! ###
2024-02-05 00:52:29,857 (conn.py 74) INFO ::: Date 2024-02-03 is weekend. Return empty dataframe.
2024-02-05 00:52:29,858 (conn.py 71) INFO ::: ### drv.fut_bydd_trd data in 2024-02-04 call starts! ###
2024-02-05 00:52:29,858 (conn.py 74) INFO ::: Date 2024-02-04 is weekend. Return empty dataframe.
2024-02-05 00:52:29,858 (conn.py 71) INFO ::: ### drv.fut_bydd_trd data in 2024-02-05 call starts! ###
2024-02-05 00:52:29,930 (conn.py 80) WARNING ::: data from api is empty.
2024-02-05 00:52:29,930 (processing.py 9) INFO ::: drv.future_kospi upload starts!
2024-02-05 00:52:29,932 (conn.py 71) INFO ::: ### drv.eqsfu_stk_bydd_trd data in 2024-02-03 call starts! ###
2024-02-05 00:52:29,932 (conn.py 74) INFO ::: Date 2024-02-03 is weekend. Return empty dataframe.
2024-02-05 00:52:29,932 (conn.py 71) INFO ::: ##

KeyboardInterrupt: 

# 2. 주식

## 1. idx

In [ ]:
import pandas as pd
import json
import requests


In [4]:
pd.set_option('display.max_rows', 100)

NameError: name 'pd' is not defined

In [15]:
o = json.loads(requests.get(f"https://data-dbg.krx.co.kr/svc/apis/{'gen'}/{conf['gen']['emit']['detail_url']}?basDd=20150113", 
             headers ={
        'AUTH_KEY': os.environ['auth_key']
        }).text)['OutBlock_1']

o = pd.DataFrame(o)
o

,BAS_DD,ISU_CD,ISU_NM,TDD_CLSPRC,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL
0,20150113,05001150,KAU15,9500,860,9.95,9500,9500,9500,50,475000
1,20150113,05001160,KAU16,7860,0,0.00,0,0,0,0,0
2,20150113,05001170,KAU17,7860,0,0.00,0,0,0,0,0


In [8]:

o = json.loads(requests.get(f"https://data-dbg.krx.co.kr/svc/apis/{sub_url}/{conf[sub_url][data_name]['detail_url']}?basDd=20190301", 
             headers ={
        'AUTH_KEY': os.environ['auth_key']
        }).text)['OutBlock_1']

o = pd.DataFrame(o)
o

,BAS_DD,ISUR_NM,ISU_CD,SRI_BND_TP_NM,ISU_NM,LIST_DD,ISU_DD,REDMPT_DD,ISU_RT,ISU_AMT,LIST_AMT,BND_TP_NM
0,20240128,신한은행,KR6000011B58,녹색채권,신한은행25-05-이10갑후(녹),20210507,20210506,20310506,2.58000,400000000000,400000000000,회사채
1,20240128,신한은행,KR6000012BB1,녹색채권,신한은행25-11-이10갑후(녹),20211110,20211109,20311109,2.84000,260000000000,260000000000,회사채
2,20240128,신한은행,KR6000017D84,녹색채권,신한은행27-08-이-3-A(녹),20230824,20230823,20260823,4.22000,100000000000,100000000000,회사채
3,20240128,신한은행,KR600001GDC7,녹색채권,신한은행27-12-이-3-A(녹),20231227,20231226,20261226,3.68000,50000000000,50000000000,회사채
4,20240128,우리은행,KR6000031924,지속가능채권,우리은행23-02-이표(지)05갑18,20190219,20190218,20240218,2.04000,200000000000,200000000000,회사채
...,...,...,...,...,...,...,...,...,...,...,...,...
2035,20240128,예금보험공사,KR352601GD42,사회적채권,예보기금특별계정채권2023-3(사),20230407,20230406,20260406,3.60000,210000000000,210000000000,특수채
2036,20240128,예금보험공사,KR352601GD59,사회적채권,예보기금특별계정채권2023-4(사),20230504,20230503,20260503,3.70000,150000000000,150000000000,특수채
2037,20240128,예금보험공사,KR352601GD67,사회적채권,예보기금특별계정채권2023-5(사),20230703,20230630,20260630,3.95000,190000000000,190000000000,특수채
2038,20240128,예금보험공사,KR352601GDC4,사회적채권,예보기금특별계정채권2023-6(사),20231211,20231208,20251208,3.80000,70000000000,70000000000,특수채


In [14]:
import pandas as pd
import json
import requests
o = json.loads(requests.get(f"https://data-dbg.krx.co.kr/svc/apis/{sub_url}/{conf[sub_url]['bnd']['detail_url']}?basDd=20100405", 
             headers ={
        'AUTH_KEY': os.environ['auth_key']
        }).text)['OutBlock_1']

o = pd.DataFrame(o)
o

,BAS_DD,MKT_NM,ISU_CD,ISU_NM,CLSPRC,CMPPREVDD_PRC,CLSPRC_YD,OPNPRC,OPNPRC_YD,HGPRC,HGPRC_YD,LWPRC,LWPRC_YD,ACC_TRDVOL,ACC_TRDVAL
0,20100405,일반채권시장,KR1035017TC0,국고0575-1012,10407.0,,2.410,10407.0,2.410,10407.0,2.410,10407.0,2.410,10000000000,10407000000
1,20100405,일반채권시장,KR6123739049,색동이제십차유동화1-9,10009.0,8.0,9.110,10009.0,9.110,10009.0,9.110,10009.0,9.110,4600000,4604140
2,20100405,일반채권시장,KR2006014V60,부산도시철도08-06,9836.0,,4.410,9836.0,4.410,9836.0,4.410,9836.0,4.410,200000000,196720000
3,20100405,일반채권시장,KR2007014V68,대구도시철도08-06,9836.0,,4.410,9836.0,4.410,9836.0,4.410,9836.0,4.410,100000000,98360000
4,20100405,일반채권시장,KR2088012V60,경남지역개발08-06,9836.0,,4.410,9836.0,4.410,9836.0,4.410,9836.0,4.410,200000000,196720000
5,20100405,일반채권시장,KR3101111W44,통안45-10-546-24,9873.0,,2.340,9873.0,2.340,9873.0,2.340,9873.0,2.340,8969000000,8855093700
6,20100405,일반채권시장,KR3105027SC2,기업은행(신종)0612이30A-11,10000.0,-100.0,6.380,10000.0,6.380,10000.0,6.380,10000.0,6.380,49000000,49000000
7,20100405,일반채권시장,KR3462017T81,JB 우리캐피탈168,9543.0,195.0,9.220,9543.0,9.220,9543.0,9.220,9543.0,9.220,2000000000,1908600000
8,20100405,일반채권시장,KR3822047V60,우리은행(신종)12-06이30갑-20,10790.0,40.0,7.180,10790.0,7.180,10790.0,7.180,10790.0,7.180,1800000,1942200
9,20100405,일반채권시장,KR6000271W33,기아자동차275,10665.0,45.0,4.610,10600.0,4.930,10700.0,4.440,10600.0,4.930,314800000,336452000


In [36]:
conf[sub_url]['etn']['detail_url']

'etf_bydd_trd'

In [16]:
##### import pandas as pd
import json
import requests
o = json.loads(requests.get(f"https://data-dbg.krx.co.kr/svc/apis/{sub_url}/{conf[sub_url]['etn']['detail_url']}?basDd=20141118", 
             headers ={
        'AUTH_KEY': os.environ['auth_key']
        }).text)['OutBlock_1']

o = pd.DataFrame(o)
o

,BAS_DD,ISU_CD,ISU_NM,TDD_CLSPRC,CMPPREVDD_PRC,FLUC_RT,PER1SECU_INDIC_VAL,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,INDIC_VAL_AMT,LIST_SHRS,IDX_IND_NM,OBJ_STKPRC_IDX,CMPPREVDD_IDX,FLUC_RT_IDX
0,20141118,530001,Perfex 유럽 고배당 주식 ETN(H),11565,1505,14.96,10119.40,10115,11565,10115,2824,28622430,57825000000,50597000000,5000000,BNPP High Div Europe Equity TR Index,-,-,-
1,20141118,570002,TRUE 코스피 선물매도 풋매도 ETN,10005,-85,-0.84,9996.68,10005,10010,9965,465,4641735,50025000000,49983400000,5000000,코스피 커버드풋 5% OTM 선물,1205.02,-8.47,-0.70
2,20141118,570001,TRUE 코스피 선물매수 콜매도 ETN,10030,75,0.75,10027.80,9980,10050,9970,900,8997495,50150000000,50139000000,5000000,코스피 커버드콜 5% OTM 선물,954.27,7.59,0.80
3,20141118,580002,able Quant비중조절 ETN,19855,130,0.66,19892.94,19910,19915,19850,2038,40502715,49637500000,49732350000,2500000,WISE K150 Quant 지수,-,-,-
4,20141118,580001,able 코스피200선물플러스 ETN,20020,165,0.83,20044.87,20035,20055,20015,2334,46775930,50050000000,50112175000,2500000,코스피 200 선물 플러스지수,1644.50,13.20,0.81
5,20141118,550001,octo Big Vol ETN,10265,190,1.89,10241.29,10140,10290,10140,334,3411560,51325000000,51206450000,5000000,Big Vol 지수,-,-,-
6,20141118,550002,octo WISE 배당 ETN,10220,215,2.15,10193.12,10060,10220,10060,697,7079495,51100000000,50965600000,5000000,WISE 셀렉트 배당 지수,-,-,-
7,20141118,520001,대우 로우볼 ETN,10025,75,0.75,10059.58,9960,10025,9960,27,270435,20050000000,20119160000,2000000,WISE 로우볼 지수,-,-,-
8,20141118,500001,신한 K200 USD 선물 바이셀 ETN,9900,65,0.66,9883.93,9835,9910,9835,65,642445,49500000000,49419650000,5000000,K200 USD 선물 바이셀지수,988.89,3.19,0.32
9,20141118,500002,신한 USD K200 선물 바이셀 ETN,10105,-45,-0.44,10116.11,10140,10140,10075,79,797775,50525000000,50580550000,5000000,USD K200 선물 바이셀지수,974.15,-3.06,-0.31


In [8]:
##### import pandas as pd
import json
import requests
o = json.loads(requests.get(f"https://data-dbg.krx.co.kr/svc/apis/{sub_url}/{conf[sub_url]['etn']['detail_url']}?basDd=20141117", 
             headers ={
        'AUTH_KEY': os.environ['auth_key']
        }).text)['OutBlock_1']

o = pd.DataFrame(o)
o

,BAS_DD,ISU_CD,ISU_NM,TDD_CLSPRC,CMPPREVDD_PRC,FLUC_RT,PER1SECU_INDIC_VAL,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,INDIC_VAL_AMT,LIST_SHRS,IDX_IND_NM,OBJ_STKPRC_IDX,CMPPREVDD_IDX,FLUC_RT_IDX
0,20141117,530001,Perfex 유럽 고배당 주식 ETN(H),10060,-5,-0.05,10051.82,10060,10060,10055,2795,28117385,50300000000,50259100000,5000000,BNPP High Div Europe Equity TR Index,-,-,-
1,20141117,570002,TRUE 코스피 선물매도 풋매도 ETN,10090,0,0.00,10067.10,10125,10135,10000,515,5171830,50450000000,50335500000,5000000,코스피 커버드풋 5% OTM 선물,1213.49,1213.49,0.00
2,20141117,570001,TRUE 코스피 선물매수 콜매도 ETN,9955,35,0.35,9948.20,9910,9955,9880,446,4415700,49775000000,49741000000,5000000,코스피 커버드콜 5% OTM 선물,946.68,946.68,0.00
3,20141117,580002,able Quant비중조절 ETN,19725,5,0.03,19726.00,19680,19785,19680,148,2923240,49312500000,49315000000,2500000,WISE K150 Quant 지수,-,-,-
4,20141117,580001,able 코스피200선물플러스 ETN,19855,35,0.18,19884.07,19815,19875,19700,143,2837055,49637500000,49710175000,2500000,코스피 200 선물 플러스지수,1631.30,1631.30,0.00
5,20141117,550001,octo Big Vol ETN,10075,-115,-1.13,10038.23,10220,10220,10025,728,7347575,50375000000,50191150000,5000000,Big Vol 지수,-,-,-
6,20141117,550002,octo WISE 배당 ETN,10005,-80,-0.79,9965.69,10100,10105,10005,576,5802770,50025000000,49828450000,5000000,WISE 셀렉트 배당 지수,-,-,-
7,20141117,520001,대우 로우볼 ETN,9950,-50,-0.50,9960.49,10040,10040,9920,155,1548750,19900000000,19920980000,2000000,WISE 로우볼 지수,-,-,-
8,20141117,500001,신한 K200 USD 선물 바이셀 ETN,9835,70,0.72,9852.40,9715,9845,9500,245,2396535,49175000000,49262000000,5000000,K200 USD 선물 바이셀지수,985.70,985.70,0.00
9,20141117,500002,신한 USD K200 선물 바이셀 ETN,10150,-90,-0.88,10148.25,10295,10295,10150,493,5030230,50750000000,50741250000,5000000,USD K200 선물 바이셀지수,977.21,977.21,0.00


In [33]:
o[o['ISU_NM'].map(lambda x: 'ETN' in x)]

,BAS_DD,ISU_CD,ISU_NM,TDD_CLSPRC,CMPPREVDD_PRC,FLUC_RT,NAV,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,INVSTASST_NETASST_TOTAMT,LIST_SHRS,IDX_IND_NM,OBJ_STKPRC_IDX,CMPPREVDD_IDX,FLUC_RT_IDX


In [44]:
json.loads(requests.get(f"https://data-dbg.krx.co.kr/svc/apis/{sub_url}/{conf[sub_url]['etf']['detail_url']}?basDd=20100215", 
             headers ={
        'AUTH_KEY': os.environ['auth_key']
        }).text)['OutBlock_1']

[{'BAS_DD': '20100215',
  'ISU_CD': '108630',
  'ISU_NM': 'FIRST 스타우량',
  'TDD_CLSPRC': '-',
  'CMPPREVDD_PRC': '-',
  'FLUC_RT': '-',
  'NAV': '-',
  'TDD_OPNPRC': '-',
  'TDD_HGPRC': '-',
  'TDD_LWPRC': '-',
  'ACC_TRDVOL': '-',
  'ACC_TRDVAL': '-',
  'MKTCAP': '-',
  'INVSTASST_NETASST_TOTAMT': '-',
  'LIST_SHRS': '1540000',
  'IDX_IND_NM': 'MKF 스타우량',
  'OBJ_STKPRC_IDX': '-',
  'CMPPREVDD_IDX': '-',
  'FLUC_RT_IDX': '-'},
 {'BAS_DD': '20100215',
  'ISU_CD': '107560',
  'ISU_NM': 'GIANT 현대차그룹',
  'TDD_CLSPRC': '-',
  'CMPPREVDD_PRC': '-',
  'FLUC_RT': '-',
  'NAV': '-',
  'TDD_OPNPRC': '-',
  'TDD_HGPRC': '-',
  'TDD_LWPRC': '-',
  'ACC_TRDVOL': '-',
  'ACC_TRDVAL': '-',
  'MKTCAP': '-',
  'INVSTASST_NETASST_TOTAMT': '-',
  'LIST_SHRS': '1890000',
  'IDX_IND_NM': 'MKF 현대차그룹',
  'OBJ_STKPRC_IDX': '-',
  'CMPPREVDD_IDX': '-',
  'FLUC_RT_IDX': '-'},
 {'BAS_DD': '20100215',
  'ISU_CD': '120210',
  'ISU_NM': 'GREAT SRI',
  'TDD_CLSPRC': '-',
  'CMPPREVDD_PRC': '-',
  'FLUC_RT': '-',
  'N

In [27]:
import pandas as pd
p = krx_api.get_data(sub_url = sub_url, detail_url = conf[sub_url]['etf']['detail_url'], date = pd.to_datetime('2010-02-15'), rename = conf['etf']['derivative']['rename'], skip_weekend = True)

2024-02-04 12:24:50,586 (conn.py 71) INFO ::: ### etp.etf_bydd_trd data in 2010-02-15 00:00:00 call starts! ###


AttributeError: Can only use .str accessor with string values!

In [23]:
p

,기준일자,ISU_CD,ISU_NM,TDD_CLSPRC,CMPPREVDD_PRC,등락률,NAV,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,거래량,거래대금,상장시가총액,INVSTASST_NETASST_TOTAMT,LIST_SHRS,IDX_IND_NM,OBJ_STKPRC_IDX,대비,FLUC_RT_IDX,_ts
0,20100104,108630,FIRST 스타우량,17200,730,4.43,17152.21,16690,17200,16690,1108,18902390,31304000000,31217018929,1820000,MKF 스타우량,NaN,NaN,NaN,2024-02-04 12:02
1,20100104,107560,GIANT 현대차그룹,12335,100,0.82,12349.27,12235,12335,12140,34989,431667005,23313150000,23340120237,1890000,MKF 현대차그룹,NaN,NaN,NaN,2024-02-04 12:02
2,20100104,120210,GREAT SRI,7810,75,0.97,7852.85,7815,7815,7785,46552,362540215,36082200000,36280150916,4620000,사회책임투자지수,NaN,NaN,NaN,2024-02-04 12:02
3,20100104,110550,HiShares Gold,6375,-5,-0.08,6372.84,6445,6445,6345,2451,15615600,11475000000,11471116743,1800000,LBMA PM FIX - Gold,1121.50,34.00,3.13,2024-02-04 12:02
4,20100104,105270,KINDEX F15,5835,40,0.69,5873.59,0,0,0,0,0,11670000000,11747179383,2000000,MKF F15 VW,NaN,NaN,NaN,2024-02-04 12:02
5,20100104,114460,KINDEX 국고채,101485,-230,-0.23,101517.50,101530,101530,101465,200,20299550,112648350000,112684429607,1110000,KTB INDEX(시장가격),9751.34,-6.42,-0.07,2024-02-04 12:02
6,20100104,108450,KINDEX 삼성그룹주,6770,120,1.80,6875.17,6695,6785,6695,190117,1292547175,32496000000,33000829412,4800000,MKF SAMs SW,6822.74,143.21,2.14,2024-02-04 12:02
7,20100104,108440,KINDEX 코스닥스타,13285,410,3.18,13269.01,13025,13285,13025,501,6616525,15144900000,15126674910,1140000,스타지수,NaN,NaN,NaN,2024-02-04 12:02
8,20100104,105190,KINDEX200,22550,85,0.38,22631.83,22390,22585,22390,58738,1325723895,198440000000,198028494388,8800000,코스피 200,223.49,1.63,0.73,2024-02-04 12:02
9,20100104,105450,KODEX 15,3895,45,1.17,3894.54,3870,3895,3855,1014,3923830,5842500000,5841805222,1500000,MKF F15 PW,NaN,NaN,NaN,2024-02-04 12:02


In [24]:
p.to_sql(conf['idx']['derivative']['table_name'], schema = conf['schema_name'], con = engine, index = False, if_exists = "append")

3

In [2]:
from datetime import datetime
datetime.astimezone

SyntaxError: invalid syntax (2849523136.py, line 2)